## **Lab2 - Data Collection and Pre-Processing**

**Name:** Mandeep Singh Brar  
**ID Number:** 8989367  
**Course Name:** Machine Learning Programming  
**Course Code:** PROG8245

### **Step 1:** Hello, Data!

In [1154]:
# Importing necessary libraries

import pandas as pd     # pandas is used for data loading, cleaning, and analysis
import numpy as np      # numpy is used for efficient numerical operations (e.g., for missing values)
import matplotlib.pyplot as plt  # matplotlib is used for creating data visualizations like plots and charts
import seaborn as sns   # seaborn is a statistical data visualization, great for advanced visualizations

# Load the dataset
df = pd.read_csv('c:/Users/singh/Desktop/500 Sales Records.csv')

# Display the first 3 rows
df.head(3)

,Order ID,Order Date,Customer ID,Product ID,Product Name,Category,Sales,Quantity,City
0,CA-2016-152156,11/8/2016,CG-12520,FUR-BO-10001798,Bush Somerset Collection Bookcase,Furniture,261.96,2.0,Henderson
1,CA-2016-152156,11/8/2016,CG-12520,FUR-CH-10000454,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,731.94,3.0,Henderson
2,CA-2016-138688,6/12/2016,DV-13045,OFF-LA-10000240,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,14.62,2.0,Los Angeles


### **Step 2:** Pick the Right Container

**Dict** stores sales data flexibly but can be messy. **namedtuple** is cleaner for fixed columns. **class** is best if we want to add methods like profit calculation.

**Conclusion:**
For above dataset, a custom class is most useful if we are planning to add data validation or extra logic. For simply storing and accessing rows, a namedtuple is clean and efficient.

### **Step 3:** Transaction Class and OO Data Structure

In [1155]:
from datetime import datetime # For converting date strings into Python date objects
from typing import Optional # For type hinting optional fields in the Transaction class

class Transaction:
    def __init__(ecommerce, 
                 order_id, 
                 order_date, 
                 customer_id, 
                 product_id, 
                 product_name, 
                 category, 
                 sales, 
                 quantity, 
                 city):
        
        ecommerce.order_id = order_id
        ecommerce.order_date = datetime.strptime(order_date, '%m/%d/%Y')
        ecommerce.customer_id = customer_id
        ecommerce.product_id = product_id
        ecommerce.product_name = product_name
        ecommerce.category = category
        ecommerce.sales = float(sales)
        ecommerce.quantity = int(quantity)
        ecommerce.city = city

    def __sale__(ecommerce):
        return f"Transaction({ecommerce.order_id}, {ecommerce.sales}, {ecommerce.city})"

print("Transaction class defined.")


Transaction class defined.


### **Step 4:** Bulk Loader

In [1156]:
from typing import List

# Function to load transactions from a DataFrame
def load_transactions(df) -> List[Transaction]:
    transactions = [] # Empty list to store Transaction objects

    # Loop through each row in the DataFrame
    for _, row in df.iterrows():

        # Create a Transaction object from row data
        transactions.append(Transaction(
            row['Order ID'],
            row['Order Date'],
            row['Customer ID'],
            row['Product ID'],
            row['Product Name'],
            row['Category'],
            row['Sales'],
            row['Quantity'],
            row['City']
        ))
    return transactions

transactions = load_transactions(df)
print(f"Loaded {len(transactions)} transactions.")

Loaded 500 transactions.


### **Step 5:** Quick Profiling

In [1157]:
import math

# Collect all sales values and unique cities from the transactions list.
sales_values = [t.sales for t in transactions]
unique_cities = {t.city for t in transactions}

# Calculate min, mean, max sales.
min_sales = min(sales_values)
mean_sales = sum(sales_values) / len(sales_values)
max_sales = max(sales_values)

# Print results to understanding the sales values and cities.
print("------ Quick Profiling ------")
print(f"Minimum Sales: {min_sales:.2f}")
print(f"Mean Sales: {mean_sales:.2f}")
print(f"Maximum Sales: {max_sales:.2f}")
print(f"Number of Unique Cities: {len(unique_cities)}")
print(f"Cities Name: {list(unique_cities)[:100]}")

------ Quick Profiling ------
Minimum Sales: 1.25
Mean Sales: 258.85
Maximum Sales: 8159.95
Number of Unique Cities: 117
Cities Name: ['Tampa', 'West Jordan', 'Columbia', 'Henderson', 'Amarillo', 'Redlands', 'Canton', 'Springfield', 'Fort Worth', 'Taylor', 'Hamilton', 'Naperville', 'Des Moines', 'Phoenix', 'Akron', 'Eagan', 'Houston', 'Louisville', 'Morristown', 'Durham', 'Gastonia', 'Bloomington', 'Jacksonville', 'Huntington Beach', 'Seattle', 'Saint Petersburg', 'Miami', 'Hesperia', 'Richardson', 'Newark', 'Inglewood', 'Costa Mesa', 'Tucson', 'Quincy', 'New York City', 'Roseville', 'Dallas', 'Hackensack', 'Rochester', 'Portland', 'Austin', 'Colorado Springs', 'Edmond', 'Belleville', 'Memphis', 'San Francisco', 'Whittier', 'Gilbert', 'Peoria', 'New Rochelle', 'Concord', 'Chapel Hill', 'Richmond', 'Saint Paul', 'Lakeville', 'Layton', 'Norman', 'Fremont', 'Jackson', 'Arlington', 'Manchester', 'Fayetteville', 'Bristol', 'Franklin', 'Troy', 'Carlsbad', 'Grand Prairie', 'Lindenhurst', 'Chi

### **Step 6:** Spot the Grime

The major purpose of the spot the grime to identify the thress types of dirty data in the dataset:
- Negative sales value
- Non-positive quantity
- Missing values

In [1158]:
# List to store having details of all dirty data cases found
dirty_cases = []

# Loop through each transaction object in the transactions list
for t in transactions:
    # Checking for negative sales vales
    if t.sales < 0:
        dirty_cases.append({
            'issue': 'Negative Sales Values',
            'order_id': t.order_id,
            'sales': t.sales,
            'city': t.city
        })
        
    # Checking for zero or negative quantity
    if t.quantity <= 0:
        dirty_cases.append({
            'issue': 'Non-positive Quantity',
            'order_id': t.order_id,
            'quantity': t.quantity,
            'city': t.city
        })
    
    # Checking for missing, null and invalid city names
    if not isinstance(t.city, str) or not t.city.strip():
        dirty_cases.append({
            'issue': 'Missing values',
            'order_id': t.order_id,
            'city': t.city,
            'sales': t.sales
        })

# Print a summary of findings
print("----- Dirty value data cases found -----")
print(f"Total dirty data cases: {len(dirty_cases)}\n")

# Show up to 5 sample dirty data cases for quick review
for i, case in enumerate(dirty_cases[:5]):
    print(f"Case {i+1}: {case}")

----- Dirty value data cases found -----
Total dirty data cases: 0



### **Step 7:** Cleaning Rules

In [1159]:
def clean(transactions):
    
    before_count = len(transactions)

    cleaned_transactions = []
    for t in transactions:
        if t.sales < 0:
            continue
        if t.quantity <= 0:
            continue
        if not isinstance(t.city, str) or not t.city.strip():
            continue
        cleaned_transactions.append(t)
    
    after_count = len(cleaned_transactions)
    print("----- Data Cleaning Report -----")
    print(f"Records before cleaning: {before_count}")
    print(f"Records after cleaning:  {after_count}")
    print(f"Records removed:         {before_count - after_count}")

    return cleaned_transactions

# Usage:
transactions_clean = clean(transactions)

----- Data Cleaning Report -----
Records before cleaning: 500
Records after cleaning:  500
Records removed:         0


### **Step 8:** Transformations

Extract a numeric discount from product_name if present in the form '10%OFF'.
Returns the integer discount (e.g., 10) or 0 if not found.

In [1160]:
def parse_discount(product_name):
    
    for word in product_name.split():
        if word.endswith('%OFF'):
            try:
                return int(word.replace('%OFF', ''))
            except ValueError:
                continue
    return 0

for t in transactions_clean:
    t.discount = parse_discount(t.product_name)

### **Step 9:** Feature Engineering

In [1161]:
from datetime import datetime

today = datetime.today()
for t in transactions_clean:
    # Difference between in now days and order_date
    t.days_since_purchase = (today - t.order_date).days

print("\n------ Days since purchase ------")
for t in transactions_clean[:5]:
    print(f"Order Date: {t.order_date.date()}, Days Since Purchase: {t.days_since_purchase}")


------ Days since purchase ------
Order Date: 2016-11-08, Days Since Purchase: 3122
Order Date: 2016-11-08, Days Since Purchase: 3122
Order Date: 2016-06-12, Days Since Purchase: 3271
Order Date: 2015-10-11, Days Since Purchase: 3516
Order Date: 2015-10-11, Days Since Purchase: 3516


### **Step 10:** Mini-Aggregation

In [1162]:
from collections import defaultdict

# Calculate total sales per city
city_sales = defaultdict(float)
for t in transactions_clean:
    city_sales[t.city] += t.sales

# Sort cities by sales in descending order and take the top 10
top_10_cities = sorted(city_sales.items(), key=lambda x: x[1], reverse=True)[:10]

print("\n------ Top 10 Cities by Total sales ------")
for city, sales in top_10_cities:
    print(f"{city}: ${sales:.2f}")


------ Top 10 Cities by Total sales ------
New York City: $15037.80
Houston: $11566.00
San Antonio: $10546.57
Los Angeles: $8991.52
San Francisco: $8308.86
Chicago: $6762.27
Philadelphia: $6264.84
Franklin: $4181.40
Lakeville: $3745.63
San Diego: $3364.07


### **Step 11:** Serialization Checkpoint

In [1163]:
!pip install pyarrow



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from pandas.io.parquet import to_parquet

# Convert back to DataFrame for saving (best for parquet/json)
df_cleaned = pd.DataFrame([t.__dict__ for t in transactions_clean])
df_cleaned.to_json("final_cleaned_transactions_data.json", orient="records", lines=True)
df_cleaned.to_parquet("final_cleaned_transactions_data.parquet")
print("JSON and Parquet files saved successfully.")

Serialized to JSON and Parquet.


### **Step 12:** Soft Interview Reflection

I have utilised OOP and the Transaction class to organise the data pipeline effectively, rendering data cleaning, transformation, and feature engineering modular and scalable. This method facilitates modifications to business logic or data structure in a singular location, enhancing maintainability and code reusability. Iterative processes—such as the incorporation of new fields or the refinement of rules—become straightforward, transparent, and resilient.

### **Data-Dictionary Section**

In [1165]:
import pandas as pd

# Load primary and secondary files
primary = df  # Assuming the primary file is the main sales records
secondary = pd.read_csv('c:/Users/singh/Desktop/secondary_product_catalogue.csv', nrows=1) # Secondary file for additional product details

# Define the data dictionary directly
data_dictionary = [
    ["Order ID", "string", "Unique identifier for each order", "Primary"],
    ["Order Date", "date", "Date the order was placed", "Primary"],
    ["Customer ID", "string", "Unique customer identifier", "Primary"],
    ["Product ID", "string", "Unique product identifier", "Primary"],
    ["Product Name", "string", "Name of the product purchased", "Primary"],
    ["Category", "string", "Product category", "Primary"],
    ["Sales", "float", "Sale amount for the order line", "Primary"],
    ["Quantity", "int", "Number of units ordered", "Primary"],
    ["City", "string", "City where order was shipped", "Primary"],
    ["Sub-Category", "string", "Detailed product sub-category", "Secondary"],
    ["Discount", "float", "Discount applied to the product", "Secondary"],
    ["Profit", "float", "Profit earned from the product", "Secondary"],
    ["Region", "string", "Sales region of the product", "Secondary"],
    ["State", "string", "State where the order was shipped", "Secondary"],
    ["days_since_purchase", "int", "Days since the order date", "Engineered"],
]

# Create and display the DataFrame
df_dictionary = pd.DataFrame(data_dictionary, columns=["Field", "Type", "Description", "Source"])
df_dictionary


,Field,Type,Description,Source
0,Order ID,string,Unique identifier for each order,Primary
1,Order Date,date,Date the order was placed,Primary
2,Customer ID,string,Unique customer identifier,Primary
3,Product ID,string,Unique product identifier,Primary
4,Product Name,string,Name of the product purchased,Primary
5,Category,string,Product category,Primary
6,Sales,float,Sale amount for the order line,Primary
7,Quantity,int,Number of units ordered,Primary
8,City,string,City where order was shipped,Primary
9,Sub-Category,string,Detailed product sub-category,Secondary


### Data Dictonary Section as Markdown

| Field              | Type   | Description                         | Source     |
|--------------------|--------|-------------------------------------|------------|
| Order ID           | string | Unique identifier for each order    | Primary    |
| Order Date         | date   | Date the order was placed           | Primary    |
| Customer ID        | string | Unique customer identifier          | Primary    |
| Product ID         | string | Unique product identifier           | Primary    |
| Product Name       | string | Name of the product purchased       | Primary    |
| Category           | string | Product category                    | Primary    |
| Sales              | float  | Sale amount for the order line      | Primary    |
| Quantity           | int    | Number of units ordered             | Primary    |
| City               | string | City where order was shipped        | Primary    |
| Sub-Category       | string | Detailed product sub-category       | Secondary  |
| Discount           | float  | Discount applied to the product     | Secondary  |
| Profit             | float  | Profit earned from the product      | Secondary  |
| Region             | string | Sales region of the product         | Secondary  |
| State              | string | State where the order was shipped   | Secondary  |
| days_since_purchase| int    | Days since the order date           | Engineered |


### Reference

I have taken the reference of chatgpt and copilot for helping me in understanding and structuring the code to write the program. I have rewrite and reviewed the completed code by my own. I took the help of the references to increase my learning level, coding the transaction, clean to fulfill my assignment requirements.  

